[메모]

(210, 164, 74) 플레이어 <br>
(187, 187, 53) 대기 플레이어 <br>
(0, 28, 136) 도로 <br>
(228, 111, 111) 벽 & 과자 <br>
(198, 89, 179) 분홍 유령 <br>
(180, 122, 48) 노랑 유령 <br>
(200, 72, 27) 빨강 유령 <br>
(84, 184, 153) 하늘 유령 <br>
(184, 50, 50) 체리 <br>
(195, 144, 61) 점수<br>

게임 자체에서 유령들이 지맘대로 움직임

In [1]:
import gym

env = gym.make('MsPacman-v0')
state = env.reset()

for _ in range(10000):
    env.render()
    observation, reward, done, info = env.step(env.action_space.sample())
    
    print('reward : ' , reward) 
    print('done : ' ,done)
    print('info : ', info)
    print(observation.shape, observation)

C:\Users\laststar\anaconda3\envs\rl-learn\lib\site-packages\gym\envs\atari\environment.py:269: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  "We strongly suggest supplying `render_mode` when "
C:\Users\laststar\anaconda3\envs\rl-learn\lib\site-packages\pyglet\image\codecs\wic.py:434: UserWarning: [WinError -2147417850] 스레드 모드가 설정된 후에는 바꿀 수 없습니다
  warnings.warn(str(err))


In [10]:
print('reward : ' , reward) 
print('done : ' ,done)
print('info : ', info)
print(observation.shape, observation)

reward :  0.0
done :  True
info :  {'lives': 0, 'episode_frame_number': 1985, 'frame_number': 1985, 'TimeLimit.truncated': False}
(210, 160, 3) [[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[228 111 111]
  [228 111 111]
  [228 111 111]
  ...
  [228 111 111]
  [228 111 111]
  [228 111 111]]

 [[228 111 111]
  [228 111 111]
  [228 111 111]
  ...
  [228 111 111]
  [228 111 111]
  [228 111 111]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]]


In [ ]:
import gym
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
def colored(r, g, b, text):
    return '\033[1m' + "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)

In [ ]:
env = gym.make('MsPacman-v0')
state = env.reset()

In [ ]:
rgb_images = []
rgb_arrays = []

In [ ]:
for _ in range(1000):
    image = env.render(mode='rgb_array')
    observation, reward, done, info = env.step(env.action_space.sample())
    rgb_images.append(image)
    rgb_arrays.append(observation)

In [ ]:
rgb_images

In [ ]:
info

In [ ]:
len(rgb_images)

In [ ]:
rgb = rgb_images[0]

In [ ]:
plt.imshow(rgb)

In [ ]:
rgb.shape

In [ ]:
color_unique = rgb.reshape((-1,3))

In [ ]:
for code in np.unique(color_unique, axis=0):
    r = code[0]
    g = code[1]
    b = code[2]
    
    print(colored(r,g,b, f"{(r,g,b)}  색상"))

### Data Preprocess

In [ ]:
df = pd.DataFrame(rgb.reshape(-1,3), columns=['R', 'G', 'B'])

In [ ]:
df['x'] = df.apply(lambda row: (int(int(row.name % 160))), axis=1)
df['y'] = df.apply(lambda row: (int(int(row.name) / 160)), axis=1)
df['pos'] = df.apply(lambda row: (row['x'], row['y']), axis=1)
df['RGB'] = df.apply(lambda row: (row['R'], row['G'], row['B']), axis=1)

In [ ]:
# 전처리 time 행 열 R G B

In [ ]:
df

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(x='RGB', data=df, color='black')

### Black

In [ ]:
black = df[df['RGB'] == (0,0,0)]

In [ ]:
black['y'].unique()

In [ ]:
black[(black.y > 0) & (black.y < 172)]['x'].unique()

black은 y가 0, 172 이상인 경우 존재

### Ghost 

#### Sky Blue Ghost

In [ ]:
sky_blue_ghost = df[df['RGB'] == (84, 184, 153)]
sky_blue_ghost.head()

In [ ]:
print(f"X => {sky_blue_ghost['x'].min()} ~ {sky_blue_ghost['x'].max()}")
print(f"Y => {sky_blue_ghost['y'].min()} ~ {sky_blue_ghost['y'].max()}")

size 8 X 10

#### PINK

In [ ]:
pink_ghost = df[df['RGB'] == (198, 89, 179)]
pink_ghost.head()

In [ ]:
print(f"X => {pink_ghost['x'].min()} ~ {pink_ghost['x'].max()}")
print(f"Y => {pink_ghost['y'].min()} ~ {pink_ghost['y'].max()}")

size 8 X 10

유령에 대해 정중앙 위치를 할 필요는 없을 것 같음 거리를 구하는 것도 아니기 때문에 위치는 하나의 상태 값
으로만 사용할 것이기 때문이다.

In [ ]:
env = gym.make('MsPacman-v0')
state = env.reset()

count = 0;

while True:
    env.render()
    observation, reward, done, info = env.step(env.action_space.sample())
    print(count, info)
    count += 1
    import time
    time.sleep(0.1)

유령의 위치뿐만 아니라 방향도??
이유는 유령이 꼭 플레이어와 닿을 수 있는 방향으론 움직이지 않기 때문
방향까진 무리고 위치만으로도 충분할 듯 한데

아이템을 제외하고 컬러수를 매긴다.

과자 포지션은 알고 있는게 좋음

전처리를 하는 이유는 디버깅을 좀 쉽게 할려는 이유도 있음 왜냐면 배열 형태론 사람이 못 알아보니깐
또 커스텀마이징도 가능함